# Last Hope and all Tasks

## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import shap

### Local or Google Colab 

In [ ]:
#Für Colab
#!git clone -b master https://github.com/HennFarr/Coins.git

In [ ]:
local_data_dir="new_extended_dataset/original"
colab_data_dir="/content/Coins/new_extended_dataset/original"

### Data

#### Clear Data

In [ ]:
num_skipped = 0
for folder_name in ("1c", "1e", "2c", "2e", "5c", "10c", "20c","50c"):
    folder_path = os.path.join(local_data_dir, folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

#### Get Data

In [ ]:
batch_size=64   
target_size = 200

In [ ]:
# Training split
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    local_data_dir,
    labels='inferred',
    label_mode='int',
    class_names=["1c", "2c", "5c", "10c", "20c", "50c", "1e", "2e"],
    validation_split=0.25,
    subset="training",
    seed=42,
    image_size=(target_size,target_size),
    batch_size=batch_size,
)

In [ ]:
# Validation split
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    local_data_dir,
    labels='inferred',
    label_mode='int',
    class_names=["1c", "2c", "5c", "10c", "20c", "50c", "1e", "2e"],
    validation_split=0.25,
    subset="validation",
    seed=42,
    image_size=(target_size,target_size),
    batch_size=batch_size,
)

## Visualisierung 

In [ ]:
class_names = train_ds.class_names

In [ ]:
# First 32 images + label of the first TRAINING batch
plt.figure(figsize=(20, 10))
for images, labels in train_ds.take(1):
  for i in range(32):
    ax = plt.subplot(4, 8, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# First 32 images + label of the first VALIDATION batch
plt.figure(figsize=(20, 10))
for images, labels in val_ds.take(1):
  for i in range(32):
    ax = plt.subplot(4, 8, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

## Model Building
>Hyperparameter Tuning missing

In [ ]:
train_ds = train_ds.prefetch(buffer_size=64)
val_ds = val_ds.prefetch(buffer_size=64)

### Data Augmentation 

In [ ]:
data_augmentation = keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(factor=0.5),
])

### Model & Layers
>specific comments missing


In [ ]:
inputs = keras.Input(shape=(target_size,target_size, 3))

x = data_augmentation(inputs)

x = layers.Rescaling(1.0 / 255)(x)

x = layers.Conv2D(filters=16, kernel_size=3, strides=2, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)

x = layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)

previous_block_activation = x  # Set aside residual

for n_filters in [64, 128, 256, 512]:
    x = layers.Conv2D(filters=n_filters, kernel_size=3, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(filters=n_filters, kernel_size=3, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(pool_size=3, strides=2, padding="same")(x)

    # Project residual
    residual = layers.Conv2D(n_filters, 1, strides=2, padding="same")(previous_block_activation)
    x = layers.add([x, residual])  # Add back residual
    previous_block_activation = x  # Set aside next residual


x = layers.Conv2D(filters=728, kernel_size=3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)


x = layers.SpatialDropout2D(0.5)(x)
#Flatten hat in den vorherigen Modellen bessere Ergebnisse erzeugt
#x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten()(x)

outputs = layers.Dense(8, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

In [ ]:
#keras.utils.plot_model(model, show_shapes=True)

### Callback
- Saving best Model 
- Early Stopping not suitable (s. history below)

In [ ]:
local_model_dir = "Models/advanced_model"
colab_model_dir = "/content/Coins/Models/advance_model"
callbacks = [
    keras.callbacks.ModelCheckpoint(colab_model_dir, save_best_only=True),
    #keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0.01, patience=4)
]

### Compiling
- 'categorical_crossentropy' works on one-hot encoded label
- 'sparse_categorical_crossentropy' works on integer label

In [ ]:
model.compile(optimizer="adam",
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

## Model Training
>Needs to be done in Google Colab

In [ ]:
epochs=50

history = model.fit(
    train_ds, 
    epochs=epochs, 
    callbacks=callbacks, 
    validation_data=val_ds,
)

### Training history

In [ ]:
epochs_range = range(epochs)

plt.plot(epochs_range, history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs_range, history.history['val_accuracy'], label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show

### Loading Model
> Loading Model in local environment for later evaluation

In [ ]:
model_loaded = keras.models.load_model(local_model_dir)

In [ ]:
val_loss, val_acc = model_loaded.evaluate(val_ds)

## Model Evaluation

## Explainable AI

### CNN Filters

### Feature Maps

### SHAP